In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import time

time.sleep(2*60)
targets = ['clear', 'sunsilk', 'lifebuoy', 'tresemme', 'ponds',"pond's", 'pond',
           'glow &', 'vaseline', 'pepsodent pasta gigi',
           'close up', 'lifebuoy sabun mandi', 'lux', 'rexona', 'axe', 'molto', 'sunlight',
           'wipol', 'vixal', 'royco', 'bango kecap', 'sariwangi', 'sarimurni', 'buavita',
           'head & shoulders', 'pantene', 'zinc', 'garnier', 'nivea',
           'marina', 'ciptadent pasta gigi', 'nuvo', 'giv', 'posh',
           'so klin',  'downy pelembut', 'mama lemon', 'super sol',
           'yuri porstex', 'masako', 'molto pewangi',
           'sedaap kecap', 'abc kecap manis', 'sosro teh', 'tong tji', 'teh bendera',
           'country choice', 'citra', 'super pel', 'rinso', 'fair']


In [2]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br, zstd',
    'X-Requested-With': 'XMLHttpRequest',
    'x-api-key': '29ea3dcf-67c3-45f6-b5c4-d2628f7e09fa',
    'Origin': 'https://shop.tiptop.co.id',
    'Connection': 'keep-alive',
    'Referer': 'https://shop.tiptop.co.id/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'If-None-Match': 'W/17183-jpdI/sqJpt20+d2buSTmZacc8PE',
}
def fetch_data(target, outlet_id):
    params = {
        'name': target,
        'outletId': outlet_id,
    }

    response = requests.get('https://api.tiptop.co.id/api/web//product-search', params=params, headers=headers, verify=False)
    return response.json()


In [3]:
def extract_product_info(data):
    product_info = []
    for product in data["data"]:
        product_name = product["name"]
        for stock in product["info_product"]["pricing_stock"]:
            size = stock["name"]
            for outlet in stock["pricing_stock_outlet"]:
                price = outlet["price"]
                discount_price = outlet.get("discount_price")
                
                product_info.append({
                    'productName': product_name +" "+ size,
                    'basePrice': price,
                    'finalPrice': discount_price if discount_price is not None else price
                })
    return product_info


In [4]:
import pandas as pd
promo_sku = pd.DataFrame(
    columns=["productName", "basePrice", "finalPrice","location"])

# Create a list to store the processed rows
data_list = []
location = {"Pondok Gede":'63cc7d04e5df8b2ef7ee517a', "Ciputat":'63c4b0eceb09e61d3a2eeb4d'}
for loc, outlet_id in location.items():
    for target in targets:
        data = fetch_data(target, outlet_id)
        product_info = extract_product_info(data)
        for item in product_info:
            item['location'] = loc
            data_list.append(item)
        promo_sku = pd.concat([promo_sku, pd.DataFrame(product_info)], ignore_index=True)

promo_sku.drop_duplicates(subset=['productName','location'], keep='first', inplace=True)
file_name = f"../tiptop/TIPTOP_{datetime.now().strftime('%y%m%d')}.xlsx"
promo_sku.to_excel(file_name, index=False)
print(promo_sku)

C:\Users\Christopher Bagas\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.tiptop.co.id'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Christopher Bagas\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.tiptop.co.id'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Christopher Bagas\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.tiptop.co.id'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usa

                                            productName basePrice finalPrice  \
0                        ACNEMED CLEAR FACIAL S.MASK NS     22875      22875   
1     ACNES FACE CARE POINT CLEAR MINI ROLL-ON DESIG...     25700      25700   
2                             AUTO GARD CLEAR 300+100ML     55950      55950   
3                          BIORE BODY CLEAR FRESH 250ML     24000      24000   
4     BIORE BODY WASH MOISTURE CARE CLEAR FRESH REFI...     17050      17050   
...                                                 ...       ...        ...   
4163                 PIXY X UVW TWC PM FAIR YELLOW 12GR     33450      33450   
4164  VIVA QUEEN FACE FLAWLESS FOUNDATION PERFECT ST...     65900      65900   
4165  VIVA QUEEN LIP COLOR PERFECT MATTELIZED LIP CR...     46500      46500   
4166  VIVA QUEEN PERFECTION NATURAL BRIGHT LOOSE POW...     42250      42250   
4167                 ZZPIXY RF UVW TWC PM FAIR YLW 12GR     21800      21800   

         location  
0     Pondok Gede  